<a href="https://colab.research.google.com/github/delemarchand2020/DeepLearning/blob/main/Projet_G%C3%A9n%C3%A9ration_de_sc%C3%A9narios_et_renforcement_Denis_Lemarchand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation des librairies

In [ ]:
!pip install chess

In [ ]:
!pip install stockfish

In [ ]:
!wget https://www.dropbox.com/sh/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1gnoo7zvmhn35gUy093Eltw6a-PWOIpwz' -O stockfish-5-linux.zip

In [ ]:
!unzip -o stockfish-5-linux.zip

In [ ]:
!chmod +x stockfish-5-linux/Linux/stockfish_14053109_x64

In [ ]:
!ls -l stockfish-5-linux/Linux/stockfish_14053109_x64

In [ ]:
!pip install stable-baselines3[extra]

##Tests python-chess

In [ ]:
import chess
import chess.engine

engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish-5-linux/Linux/stockfish_14053109_x64")

max_iter = 1000
k = 0

board = chess.Board()
while not board.is_game_over() and k < max_iter:
    if(board.turn): #blanc
      white_player = engine.play(board, chess.engine.Limit(time=0.01))#, depth=2, nodes=5))
      board.push(white_player.move)
    else:
      black_player = engine.play(board, chess.engine.Limit(time=0.001))
      board.push(black_player.move)
    k += 1

engine.quit()

print(board.status, k, board.result())
board_gagnant = board
board_gagnant

##Fonctions utilitaires (issues de RLC)

In [ ]:
def get_project_legal_moves(board):
        """
        Create a mask of legal actions
        Returns: np.ndarray with shape (64,64)
        """
        action_space = np.zeros(shape=(64, 64))
        moves = [[x.from_square, x.to_square] for x in board.generate_legal_moves()]
        for move in moves:
            action_space[move[0], move[1]] = 1
        return action_space

In [ ]:
def get_layer_board(board):
        """
        Initalize the numerical representation of the environment
        Returns:
        """
        layer_board = np.zeros(shape=(8, 8, 8))
        for i in range(64):
            row = i // 8
            col = i % 8
            piece = board.piece_at(i)
            if piece == None:
                continue
            elif piece.symbol().isupper():
                sign = 1
            else:
                sign = -1
            layer = mapper[piece.symbol()]
            layer_board[layer, row, col] = sign
        if board.turn:
            layer_board[6, :, :] = 1 / board.fullmove_number
        if board.can_claim_draw():
            layer_board[7, :, :] = 1
        return layer_board

In [ ]:
def next_white_move(agent, board, debug=False, best_probs=False):        
  states = []
  actions = []
  rewards = []
  action_spaces = []

  state = get_layer_board(board)
  action_space = get_project_legal_moves(board) # The environment determines which moves are legal
  action_probs = agent.model.predict([np.expand_dims(state, axis=0),
                                       np.zeros((1, 1)),
                                       action_space.reshape(1, 4096)])
  action_probs = action_probs / action_probs.sum()
  move = np.random.choice(range(4096), p=np.squeeze(action_probs))
  if(debug):
    print(move, np.argmax(action_probs, axis=1)[0])
  if(best_probs):
    move = np.argmax(action_probs, axis=1)[0]
  move_from = move // 64
  move_to = move % 64
  moves = [x for x in board.generate_legal_moves() if \
           x.from_square == move_from and x.to_square == move_to]
  assert len(moves) > 0  # should not be possible
  if len(moves) > 1:
    move = np.random.choice(moves)  # If there are multiple max-moves, pick a random one.
  elif len(moves) == 1:
    move = moves[0]
  return move      

In [ ]:
def get_random_action(board):
  legal_moves = [x for x in board.generate_legal_moves()]
  legal_moves = np.random.choice(legal_moves)
  return legal_moves

#Génération des configurations de l’échiquier avec un $β$-VAE

##Librairies

In [ ]:
import torch
use_gpu = False
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
print(f"Exécution sur {device}")

# Imports des bibliothèques utiles
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from matplotlib import pyplot as plt

##Hyperparamètres

Wout = (Win - K + 2P)/S + 1

Hout = (Hin - K + 2P)/S + 1

K:Kernel

P:Padding

S:Stride

In [ ]:
Kernel_size = 3
Stride_size = 1
Padding_size = 1
N_channels_1 = 11
N_channels_2 = 11

seuil_proba_prediction = 0.2

W_in = 8
H_in = 8

W_out = (W_in - Kernel_size + 2*Padding_size)/Stride_size + 1
H_out = (H_in - Kernel_size + 2*Padding_size)/Stride_size + 1

print(W_out,H_out)

W_out = int(W_out)
H_out = int(H_out)

In [ ]:
beta = 2
latent_dimension = 100
batch_size = 128

N_epochs = 20
learning_rate = 0.001

In [ ]:
Taille_DA = 20000
Taille_DT = 4000

##Conversion échiquier <-> numpy board

In [ ]:
def board_to_numpy(board):
        """
        Initalize the numerical representation of the environment
        Returns:
        """
        layer_board = np.zeros(shape=(6, 8, 8))
        for i in range(64):
            row = i // 8
            col = i % 8
            piece = board.piece_at(i)
            if piece == None:
                continue
            elif piece.symbol().isupper():
                sign = 1
            else:
                sign = -1
            layer = mapper[piece.symbol()]
            layer_board[layer, row, col] = sign
        return layer_board


In [ ]:
mapper = {}
#pieces noires
mapper["p"] = 0 #pion 
mapper["r"] = 1 #tour
mapper["n"] = 2 #cavalier
mapper["b"] = 3 #fou
mapper["q"] = 4 #reine
mapper["k"] = 5 #roi
#pieces blanches
mapper["P"] = 0
mapper["R"] = 1
mapper["N"] = 2
mapper["B"] = 3
mapper["Q"] = 4
mapper["K"] = 5

layer = {}
layer[0] = chess.PAWN
layer[1] = chess.ROOK
layer[2] = chess.KNIGHT
layer[3] = chess.BISHOP
layer[4] = chess.QUEEN
layer[5] = chess.KING

def numpy_to_board(layer_board):
    board = chess.Board.empty()
    for l in range(6):
       for row in range(8):
          for col in range(8):
             piece = layer_board[l][row][col]
             if piece == 1: #blanche
               c = chess.WHITE
             elif piece == -1: #noire
               c = chess.BLACK
             else:
               continue
             p = chess.Piece(layer[l],c)
             board.set_piece_at(row*8+col, piece=p)
    return board      

##Préparation du dataset des configurations

In [ ]:
# Configurer une instance de stockfish, une zone mémoire D pour stocker les configurations réelles d’échiquiers.
import chess
import chess.engine

class DatasetConfigBoard(object):

    def __init__(self, N=1000, N_test=200, alea=0):
        self.N = N
        self.memory_DA = torch.Tensor(size=(N,6,8,8))
        self.memory_DT = torch.Tensor(size=(N_test,6,8,8))
        self.memory_Boards = [] #pour tests
        self.N_test = N_test
        self.alea = alea

        self.stat_nb_parties = 0
        self.stat_nb_moyen_coups_par_parties = 0
        self.nb_victoires_blancs = 0
        self.nb_victoires_noirs = 0
        self.nb_fois_atteinte_max_iter = 0

        #self.nb_moyen_captures_blancs_par_parties = 0
        #self.nb_moyen_captures_noirs_par_parties = 0

    def generate(self, N, memory_D, memory_Boards, calculs_stats = False):
        engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish-5-linux/Linux/stockfish_14053109_x64")
        i = 0
        while i < N:
          max_iter = 1000
          k = 0
          board = chess.Board()
          while not board.is_game_over() and k < max_iter and i < N:
              if(board.turn): #blanc
                white_player = engine.play(board, chess.engine.Limit(time=0.01))
                board.push(white_player.move)
              else:
                black_player = engine.play(board, chess.engine.Limit(time=0.015))
                board.push(black_player.move)
              k += 1

              if np.random.uniform() > self.alea or k < 20:
                memory_D[i] = torch.Tensor(board_to_numpy(board))
                memory_Boards.append(board.copy())
                i += 1
          
          if calculs_stats:
              self.stat_nb_parties += 1
              self.stat_nb_moyen_coups_par_parties += k
                        
              if k == max_iter:
                self.nb_fois_atteinte_max_iter += 1
          
              if board.result() == "1-0":
                self.nb_victoires_blancs += 1

              if board.result() == "0-1":
                self.nb_victoires_noirs += 1

        if calculs_stats:
            self.stat_nb_moyen_coups_par_parties = self.stat_nb_moyen_coups_par_parties/self.stat_nb_parties
        engine.quit()

    def create_dataset(self):
        self.generate(self.N, self.memory_DA, self.memory_Boards, calculs_stats = True)
        self.generate(self.N_test, self.memory_DT, self.memory_Boards)
        return self.memory_DA, self.memory_DT

    def get_stats(self):
        print(f"nb_parties {self.stat_nb_parties}")
        print(f"nb_moyen_coups_par_parties {self.stat_nb_moyen_coups_par_parties}")
        print(f"nb_victoires_blancs {self.nb_victoires_blancs}")
        print(f"nb_victoires_noirs {self.nb_victoires_noirs}")
        print(f"nb_matchs_nuls {self.stat_nb_parties - self.nb_victoires_noirs - self.nb_victoires_blancs}")
        print(f"nb_fois_atteinte_max_iter {self.nb_fois_atteinte_max_iter}")

In [ ]:
dataset_config = DatasetConfigBoard(N=Taille_DA, N_test=Taille_DT, alea=0.5)
DA, DT = dataset_config.create_dataset()

In [ ]:
# normalisation : -1 -> 0, 0 -> 0.5, 1 -> 1
# normalisation(x): (x + 1)/2
def normalisation(x):
  return (x+1)/2

def de_normalisation(x):
  return (2*x - 1)

In [ ]:
DA_copy = DA
DT_copy = DT

In [ ]:
DA = normalisation(DA_copy)
DT = normalisation(DT_copy)

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(DA, batch_size=batch_size, shuffle=True)

##Implémentation $\beta$-VAE

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_dimension):
        super(Encoder, self).__init__()
        self.model = nn.Sequential(nn.Conv2d(6, N_channels_1, kernel_size=Kernel_size, stride=Stride_size, padding=Padding_size),
                                     nn.ReLU(),
                                     nn.Conv2d(N_channels_1, N_channels_2, kernel_size=Kernel_size, stride=Stride_size, padding=Padding_size),
                                     nn.ReLU(),
                                     nn.Flatten(),
                                   )
        self.linear1 = nn.Linear(in_features=(N_channels_2*W_out*H_out), out_features=latent_dimension)
        self.linear2 = nn.Linear(in_features=(N_channels_2*W_out*H_out), out_features=latent_dimension)

    def forward(self, x):
        x = self.model(x)
        x_mu = self.linear1(x)
        x_logvar = self.linear2(x)
        return x_mu, x_logvar

class Decoder(nn.Module):
    def __init__(self, latent_dimension):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(in_features=latent_dimension, out_features=N_channels_2*W_out*H_out)
        self.model = nn.Sequential(nn.ConvTranspose2d(N_channels_2, N_channels_1, kernel_size=Kernel_size, stride=Stride_size, padding=Padding_size),
                                     nn.ReLU(),
                                     nn.ConvTranspose2d(N_channels_1, 6, kernel_size=Kernel_size, stride=Stride_size, padding=Padding_size),
                                     nn.Sigmoid()
                                    )

    def forward(self, z):
        hat_x = F.relu(self.linear(z))
        hat_x = hat_x.view(-1, N_channels_2, W_out, H_out)
        hat_x = self.model(hat_x)
        return hat_x

In [ ]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dim):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        z = self.latent_sample(latent_mu, latent_logvar)
        hat_x = self.decoder(z)
        return hat_x, latent_mu, latent_logvar

    def latent_sample(self, mu, logvar):
        if self.training:
            # the reparameterization trick
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

In [ ]:
def vae_loss(hat_x, x, mu, logvar):
    reconstruction_loss = F.binary_cross_entropy(hat_x.view(-1, 6*8*8), x.view(-1, 6*8*8), reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + beta * kl_divergence

##Apprentissage $\beta$-VAE

In [ ]:
from tqdm.notebook import trange, tqdm

def train_vae(net, train_dataset, epochs=10, learning_rate=1e-3, batch_size=128, device=device):
    # Création du DataLoader pour charger les données
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # Définition de l'algorithme d'optimisation (Adam, variante de la SGD)
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate, weight_decay=1e-5)
    # Choix de la fonction de coût
    criterion = vae_loss
    # Passe le modèle en mode "apprentissage"
    net = net.to(device)
    net = net.train()

    t = trange(1, epochs + 1, desc="Entraînement du modèle")
    for epoch in t:
        avg_loss = 0.
        # Parcours du dataset pour une epoch
        for images in tqdm(train_dataloader):
            # les labels sont ignorés pour l'apprentissage de l'auto-encodeur

            images = images.to(device)
            # Calcul de la reconstruction
            reconstructions, latent_mu, latent_logvar = net(images)
            # Calcul de l'erreur
            loss = criterion(reconstructions, images, latent_mu, latent_logvar)

            # Rétropropagation du gradient
            optimizer.zero_grad()
            loss.backward()
            # Descente de gradient (une itération)
            optimizer.step()
            avg_loss += loss.item()

        avg_loss /= len(train_dataloader)
        t.set_description(f"Epoch {epoch}: loss = {avg_loss:.3f}")
    return net.to("cpu")

In [ ]:
vae = VariationalAutoencoder(latent_dimension)
train_vae(vae, DA, epochs=N_epochs, learning_rate=learning_rate, batch_size=batch_size)

##Tests de génération de configurations synthétiques

###Cas a : la reconstruction est-elle identique à l’entrée, est-elle valide ?

In [ ]:
def predict_layer_board(fake_config_board, p=0.5):
  layer_board = torch.round(fake_config_board, decimals=2)
  layer_board[layer_board>p] = 1
  layer_board[layer_board<-p] = -1
  layer_board[torch.abs(layer_board)!=1] = 0
  return layer_board

In [ ]:
vae.eval()
reconstruction,_,_ = vae(DT)

In [ ]:
reconstruction_config_board = de_normalisation(reconstruction.detach())
reconstruction_layer_board = predict_layer_board(reconstruction_config_board, p=seuil_proba_prediction)

In [ ]:
n = 354
b = numpy_to_board(reconstruction_layer_board[n].numpy())
print(b.status())
b

In [ ]:
dataset_config.memory_Boards[len(DA)+n]

In [ ]:
print(b.board_fen())
print(dataset_config.memory_Boards[len(DA)+n].board_fen())

In [ ]:
cas_a_invalide = 0
cas_a_invalide_DT = 0
cas_a_different = 0

invalide_board_DT = []

for i in range(len(reconstruction_layer_board)):
  b = numpy_to_board(reconstruction_layer_board[i].numpy())
  b_DT = numpy_to_board(de_normalisation(DT)[i].numpy())
  cas_a_invalide += int(b.status() != chess.STATUS_VALID)
  if b_DT.status() != chess.STATUS_VALID:
    cas_a_invalide_DT += 1
    invalide_board_DT.append(b_DT)
  cas_a_different += int(b.board_fen() != dataset_config.memory_Boards[len(DA)+i].board_fen())

In [ ]:
print(invalide_board_DT[30].status())
invalide_board_DT[30]

In [ ]:
print(de_normalisation(DT)[0][0])
b_DT = numpy_to_board(de_normalisation(DT)[0].numpy())
print(b_DT)
int(dataset_config.memory_Boards[len(DA)+0].status() != chess.STATUS_VALID)
b_DT

In [ ]:
dataset_config.memory_Boards[len(DA)+0]

###Cas b : est-ce une configuration synthétique valide ?

In [ ]:
vae.eval()

with torch.no_grad():

    # Échantillonnage selon une loi normale
    latent = torch.randn(Taille_DT, latent_dimension, device=device)

    # Reconstruction
    fake_config_board = de_normalisation(vae.decoder(latent).cpu())

In [ ]:
pred_layer_board = predict_layer_board(fake_config_board, p=0.21) #seuil_proba_prediction
pred_layer_board[4][0]

In [ ]:
b = numpy_to_board(pred_layer_board[int(np.random.uniform(low=0,high=len(pred_layer_board)))].numpy())
print(b.status())
b

In [ ]:
cas_b_invalide = 0
valid_board_cas_b = []

for i in range(len(pred_layer_board)):
  b = numpy_to_board(pred_layer_board[i].numpy())
  if b.status() != chess.STATUS_VALID:
    cas_b_invalide += 1
  else:
    valid_board_cas_b.append(b)

In [ ]:
b = valid_board_cas_b[int(np.random.uniform(low=0,high=len(valid_board_cas_b)))]
print(b.status())
b

In [ ]:
cas_b_empty = 0
last_i_not_empty = 0
first_i_not_empty = 0
for i in range(len(pred_layer_board)):
  b = numpy_to_board(pred_layer_board[i].numpy())
  if chess.STATUS_EMPTY in b.status():
    cas_b_empty += 1
    if first_i_not_empty == 0 and i > 1:
      first_i_not_empty = i-1
  else:
    last_i_not_empty = i

In [ ]:
b = numpy_to_board(pred_layer_board[first_i_not_empty].numpy())
print(b.status())
b

#Résultats générations configurations

In [ ]:
dataset_config.get_stats() #sur dataset apprentissage

In [ ]:
print(f"cas a : taux d'échec (validité) de {cas_a_invalide*100/len(reconstruction_layer_board)}%")
print(f"cas a : taux de différence {cas_a_different*100/len(reconstruction_layer_board)}%")
print(f"cas a : [DT]taux d'échec (validité) de {cas_a_invalide_DT*100/len(reconstruction_layer_board)}%")

In [ ]:
print(f"cas b : taux d'échec de {cas_b_invalide*100/len(pred_layer_board)}%")

#Environnement Gym pour les échecs

##Librairies

In [ ]:
import gym
from gym import spaces
import numpy as np

##Class ChessForRigEnv

In [ ]:
class ChessForRigEnv(gym.Env):
      """Environnement de jeu d'échec au format OpenAI gym"""
      metadata = {'render.modes': ['human']}

      def __init__(self):
        super(ChessForRigEnv, self).__init__()

        self.board = chess.Board()

        # Action = 1 déplacement de pièce : move from, move to
        # Cela revient à une position de départ dans l'échiquier (64 cases) : from
        # vers une autre position : to
        # donc c'est un tableau de 64x64 = 4096 positions 
        self.action_space = spaces.Discrete(4096) #spaces.Box(0, 4096, shape=(1,), dtype=np.float32) #spaces.Discrete(4096)

        # Observation = configuration échiquier
        # On reprend la représentation numérique de l'échiquier (6,8,8)
        # On met la structure à plat par contre
        self.observation_space = spaces.Box(-1, 1, shape=(6*8*8,), dtype=int)

      def reset(self,seed=None, return_info=False, options=None):
        self.board = chess.Board()
        return np.reshape(board_to_numpy(self.board),(6*8*8,))

      def step(self, action, mode_abandon=False):
          is_null_move = False
          move_from = int(action) // 64
          move_to = int(action) % 64
          moves = [x for x in self.board.generate_legal_moves() if \
                   x.from_square == move_from and x.to_square == move_to]
          if len(moves) > 1:
            move = np.random.choice(moves)  # If there are multiple max-moves, pick a random one.
          elif len(moves) == 1:
            move = moves[0]
          else:
            move = chess.Move.null() # le joueur ne sait pas quoi jouer !
            is_null_move = True
            #print("null move")

          if not is_null_move:
            piece_balance_before = self.get_material_value()
            self.board.push(move) # le joueur joue (notre agent)
            piece_balance_after = self.get_material_value()

            if not self.board.is_game_over():
              opposant_move = get_random_action(self.board)
              self.board.push(opposant_move) # l'opposant joue
              reward = self.calcul_rewards(piece_balance_after, piece_balance_before)
            else:
              reward = self.calcul_rewards(0, 0)

            done = self.board.is_game_over()
            obs = np.reshape(board_to_numpy(self.board),(6*8*8,))
          else:
            #opposant_move = get_random_action(self.board)
            #self.board.push(opposant_move) # l'opposant joue à ma place
            #if not self.board.is_game_over():
            #  opposant_move = get_random_action(self.board)
            #  self.board.push(opposant_move) # l'opposant joue son tour             
            if mode_abandon == True:
              done = True
              reward = 0
            else:
              self.board.push(chess.Move.null()) # on passe notre tour
              opposant_move = get_random_action(self.board)
              self.board.push(opposant_move) # l'opposant joue
              done = self.board.is_game_over()
              reward = 0 #on ne récompense pas notre agent !

            obs = np.reshape(board_to_numpy(self.board),(6*8*8,))

          return obs, reward, done, {}
      
      def render(self, mode='human',display=False):
        if display:
          print(self.board)
        return self.board

      def get_material_value(self):
        """
        Sums up the material balance using Reinfield values
        Returns: The material balance on the board
        """
        layer_board = board_to_numpy(self.board)

        pawns = 1 * np.sum(layer_board[0, :, :])
        rooks = 5 * np.sum(layer_board[1, :, :])
        minor = 3 * np.sum(layer_board[2:4, :, :])
        queen = 9 * np.sum(layer_board[4, :, :])

        return pawns + rooks + minor + queen

      def calcul_rewards(self, piece_balance_after, piece_balance_before):

        capture_reward = piece_balance_after - piece_balance_before
        reward = capture_reward

        if self.board.is_game_over():
          if self.board.result() == "*":
            reward = 0 + reward
          else:
            if self.board.result() == "1-0":
              reward = 10 + reward
            else:
              reward = -1 + reward

        return reward

In [ ]:
action_space = spaces.Box(0, 4096, shape=(1,), dtype=np.float64)
int(action_space.sample())

In [ ]:
np.array([1])

In [ ]:
action_space = spaces.Discrete(4096)
action_space.sample()

In [ ]:
obs_space = spaces.Box(-1, 1, shape=(6,8,8,), dtype=int)
t = obs_space.sample()
np.reshape(t,(6*8*8,))

##Test ChessForRigEnv (test avec stockfish)

In [ ]:
import chess
import chess.engine

rounds = 2
max_iter_per_round = 1000

engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish-5-linux/Linux/stockfish_14053109_x64")

env = ChessForRigEnv()

for i in range(rounds):
  obs = env.reset()
  done = False
  cumulative_rewards = 0
  k = 0
  
  while not done and k < max_iter_per_round:
    result = engine.play(env.board, chess.engine.Limit(time=0.1, depth=4, nodes=3))
    result = result.move
    #result = get_random_action(env.board)
    obs, reward, done, info = env.step(result.from_square*64+result.to_square) #env.step(np.array([result.from_square*64+result.to_square]))
    #print(env.board.status())
    cumulative_rewards = reward + cumulative_rewards
    k += 1

b = env.render(display=False)
print(k, cumulative_rewards, b.result(), b.status())
engine.quit()
b

##Utilisation TD3, A2C, DQN stable baselines avec ChessForRigEnv

In [ ]:
import gym
import numpy as np

from stable_baselines3 import TD3, A2C, DQN
#from stable_baselines3.td3.policies import MlpPolicy, CnnPolicy, MultiInputPolicy
from stable_baselines3.a2c.policies import MlpPolicy, CnnPolicy, MultiInputPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

#torch.set_default_dtype(torch.float32)

env = ChessForRigEnv()

# The noise objects for TD3
#n_actions = env.action_space.shape[-1]
#action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

#model = TD3(MlpPolicy, env, action_noise=action_noise, verbose=1)
#model = A2C("MlpPolicy", env, verbose=0)
model = DQN("MlpPolicy", env, verbose=1, batch_size=128, target_update_interval=1000, 
            exploration_initial_eps=0.5, exploration_final_eps=0.02, learning_rate=1e-5)

In [ ]:
model.learn(total_timesteps=5000000, log_interval=1000)
model.save("dqn_chess")

In [ ]:
#del model # remove to demonstrate saving and loading
#model.save("dqn_chess")
#model = TD3.load("td3_chess")

env = ChessForRigEnv()

obs = env.reset()
cumulative_rewards = 0
done = False
i = 0

while not done and i < 300:
    action, _ = model.predict(obs)
    move = action #np.array([action])
    obs, rewards, done, info = env.step(move, mode_abandon=False)
    cumulative_rewards += rewards
    i += 1

b = env.render(display=False)
print(i, action, cumulative_rewards, b.result(), b.status())
b

In [ ]:
from stable_baselines3.common import env_checker
env = ChessForRigEnv()
env_checker.check_env(env, warn=True, skip_render_check=False)

In [ ]:
obs = env.reset()
#obs

#Implementation DQN classique pour Chess Env
(reprise du code RCP211/RL/TP3)

In [ ]:
#@title Import des librairies
import os

import random
import gym
import pylab
import numpy as np
from collections import deque
from gym.wrappers import Monitor

#from IPython.display import HTML
#from IPython import display as ipythondisplay
#from pyvirtualdisplay import Display
#display = Display(visible=0, size=(1400, 900))
#display.start()

import time
from matplotlib import pyplot as plt

import pylab as pl
#from IPython import display as ipdisplay

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#@title Class DQNAgent 

class DQNAgent(object):
    def __init__(self, env, model, replay1d, update_target_model ,ddqn=False,Soft_Update=False, dueling=False,batch_size=128,gamma=0.95,memory_size=1e4):
              
        self.env = env
        self.env.seed(0)  
        self.replay1d = replay1d
        self.update_target_model = update_target_model

        self.env._max_episode_steps = 4000
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.EPISODES = 2000
        
        # Instantiate memory
        memory_size = 10000
        self.memory = deque(maxlen=65536)

        self.gamma = 0.95    # discount rate
        
        # EXPLORATION HYPERPARAMETERS for epsilon and epsilon greedy strategy
        self.epsilon = 1.0  # exploration probability at start
        self.epsilon_min = 0.01  # minimum exploration probability 
        self.epsilon_decay = 0.0005  # exponential decay rate for exploration prob
        
        self.batch_size = batch_size

        # defining model parameters
        self.ddqn = ddqn # use doudle deep q network
        self.Soft_Update = Soft_Update # use soft parameter update
        self.dueling = dueling
        self.epsilon_greedy = False # use epsilon greedy improved strategy
        
        self.TAU = .5 # target network soft update hyperparameter

        self.Save_Path = 'Models'
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.scores, self.episodes, self.average = [], [], []

        self.Model_name = os.path.join(self.Save_Path, "ChessForRig_DQN.h5")
        
        # create main model and target model
        self.model = model(in_channels=self.state_size,action_space = self.action_size, dueling = False, name = 'online model').to(device)
        self.target_model = model(in_channels=self.state_size,action_space = self.action_size, dueling = False, name = 'target model').to(device) 

        self.optimizer = optim.RMSprop(self.model.parameters(),lr=0.00025, eps=0.01, alpha=0.95)
        self.criterion = nn.MSELoss()
        self.loss = 0
        self.dd = torch.zeros(64, 4, 5, 5).to(device)
        
    # after some time interval update the target model to be the same as the online model               
                
    def remember(self, state, action, reward, next_state, done):
        experience = state, action, reward, next_state, done
        #print(f' state = {state}, action = {action}, reward = {reward}, next_state = {next_state}, done = {done}  ')
        self.memory.append((experience))
    
    def act1d(self, state, decay_step):
        # EPSILON GREEDY STRATEGY
        if self.epsilon_greedy:
        # Here we'll use an improved version of our epsilon greedy strategy for Q-learning
            explore_probability = self.epsilon_min + (self.epsilon - self.epsilon_min) * np.exp(-self.epsilon_decay * decay_step)
        # OLD EPSILON STRATEGY
        else:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= (1-self.epsilon_decay)
            explore_probability = self.epsilon
    
        if explore_probability > np.random.rand():
            # Make a random action (exploration)
            #q = random.randrange(self.action_size)
            result = get_random_action(self.env.board)
            action = result.from_square*64+result.to_square
            return action, explore_probability#int(np.argmax(q)), explore_probability

        else:
            # Get action from Q-network (exploitation)
            # Estimate the Qs values state
            # Take the biggest Q value (= the best action)
            q = self.model(torch.from_numpy(state).float().unsqueeze(0).to(device))
            return int(torch.argmax(q)), explore_probability
        
    #pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        pylab.plot(self.episodes, self.average, 'r')
        pylab.plot(self.episodes, self.scores, 'b')
        pylab.ylabel('Score', fontsize=18)
        pylab.xlabel('Steps', fontsize=18)
        dqn = 'DQN_'
        softupdate = ''
        dueling = ''
        greedy = ''
        PER = ''
        if self.ddqn: dqn = 'DDQN_'
        if self.Soft_Update: softupdate = '_soft'
        if self.dueling: dueling = '_Dueling'
        if self.epsilon_greedy: greedy = '_Greedy'
        try:
            pylab.savefig(dqn+"ChessForRig"+softupdate+dueling+greedy+PER+"_CNN.png")
        except OSError:
            pass

        return str(self.average[-1])[:5]

    
    def reset1d(self):
        return self.env.reset()

    
    def step1d(self,action):
        next_state, reward, done, info = self.env.step(action)
        return next_state, reward, done, info
    
    
    def run1d(self):
        decay_step = 0
        loss_tab = []
        dd = 0
        dd_old = 0
        
        for e in range(self.EPISODES):
            state = self.reset1d()
            done = False
            i = 0
            cum_rewards = 0
            while not done:
                decay_step += 1
                action, explore_probability = self.act1d(state, decay_step)
                next_state, reward, done, _ = self.step1d(action)

                if not done or i == self.env._max_episode_steps-1:
                    reward = reward
                else:
                    reward = -1e2
                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1
                cum_rewards += reward
                if done:
                    if e % 10==0:
                        update_target_model(self.Soft_Update,self.ddqn,self.model,self.target_model,self.TAU)
                    average = self.PlotModel(cum_rewards, e)#self.PlotModel(i, e)
                    loss_tab.append(self.loss)
                    print("episode: {}/{}, cum_rewards: {}, e: {:.2}, average: {}\n".format(e, self.EPISODES, cum_rewards, explore_probability, average))
                    if i == self.env._max_episode_steps:
                        print("Saving trained model to", self.Model_name)
                        break
                self.replay1d(self.env,self.memory,self.batch_size,self.model,self.target_model,self.ddqn,self.gamma,self.criterion,self.loss,self.optimizer)
        plt.plot(loss_tab)

In [ ]:
#@title class DQN1d pour fonction valeur
class DQN1d(nn.Module):

    def __init__(self, action_space, in_channels, dueling=False, name='NN'):
        super(DQN1d, self).__init__()
        self.name = name
        self.dueling = dueling
        self.action_space = action_space

        self.head1 = nn.Linear(in_channels, 512)
        self.head2 = nn.Linear(512, 256)
        self.head3 = nn.Linear(256, 64)
        
        if dueling:
            self.state_value = nn.Linear(64,1)
            self.action_advantage = nn.Linear(64,action_space)
        else:
            self.Q = nn.Linear(64,action_space)
            
        
    def forward(self, x):
        #print(f'{self.name} input = {x.shape}')
        x = F.relu(self.head1(x))
        x = F.relu(self.head2(x))
        x = F.relu(self.head3(x))
        
        if self.dueling:
            s = self.state_value(x).repeat((1,self.action_space))
            a = self.action_advantage(x)
            a -= torch.mean(a,dim=1, keepdim=True).repeat(1,2)
            x = s+a
        else:
            x = self.Q(x)       
        
        #print(f'{self.name} output = {x.size()}')
        return x


In [ ]:
#@title replay1d
def replay1d(env,memory,batch_size,model,target_model,ddqn,gamma,criterion,loss,optimizer):
        # Randomly sample minibatch from the deque memory
        minibatch = random.sample(memory, min(len(memory), batch_size))

        state = np.zeros([batch_size,env.observation_space.shape[0]])
        #print(state.shape)
        next_state = np.zeros([batch_size,env.observation_space.shape[0]])
        action, reward, done = [], [], []

        # do this before prediction
        # for speedup, this could be done on the tensor level
        # but easier to understand using a loop       
        for i in range(len(minibatch)):
            #print("minibatch[i][0] = ",minibatch[i][0])
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])
        # do batch prediction to save speed
        # predict Q-values for starting state using the main network
        target = model(torch.from_numpy(state).float().to(device)).detach()
        target_old = target.clone()
        # predict best action in ending state using the main network
        target_next = model(torch.from_numpy(next_state).float().to(device)).detach()
        # predict Q-values for ending state using the target network
        target_val = target_model(torch.from_numpy(next_state).float().to(device)).detach()
        for i in range(len(minibatch)):
            # correction on the Q value for the action used
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                # the key point of Double DQN
                # selection of action is from model
                # update is from target model
                if ddqn: # Double - DQN
                    # current Q Network selects the action
                    # a'_max = argmax_a' Q(s', a')
                    a =  int(torch.argmax(target_next[i]))# np.argmax(target_next[i])
                    # target Q Network evaluates the action
                    # Q_max = Q_target(s', a'_max)
                    target[i][action[i]] = reward[i] + gamma * (target_val[i][a])
                else: # Standard - DQN
                    # DQN chooses the max Q value among next actions
                    # selection and evaluation of action is on the target Q Network
                    # Q_max = max_a' Q_target(s', a')
                    target[i][action[i]] = reward[i] + gamma * target_next[i].max(0)[0].detach()

            
                
        # Train the Neural Network with batches
        # self.model.fit(state, target, batch_size=self.batch_size, verbose=0)
        pred = model(torch.from_numpy(state).float().to(device))
        loss = criterion(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
#@title update_target_model
def update_target_model(Soft_Update,ddqn,model,target_model,TAU):
        if not Soft_Update:
            
            target_model.load_state_dict(model.state_dict()) 
            return
        if Soft_Update and ddqn:    
            with torch.no_grad():
                #dict_model = dict(self.model.named_parameters())
                #dict_target = dict(self.target.named_paramters())
                for model, target in zip(model.named_parameters(), target_model.named_parameters()):
                    model_name, model_weight = model
                    target_name, target_weight = target
                    #print(f'target_weight = {target_weight.data}')
                    tmp = target_weight.data * (1-TAU) + model_weight.data * TAU
                    target_weight.data.copy_(tmp)


##Run DQN classique dans l'environnement Chess

In [ ]:
env = ChessForRigEnv()

In [ ]:
agentDQN = DQNAgent(env, DQN1d, replay1d,update_target_model)
agentDQN.run1d()

In [ ]:
torch.save(agentDQN.target_model.state_dict(), "target_model_dqn_RIG")
torch.save(agentDQN.model.state_dict(), "model_dqn_RIG")

In [ ]:
env_test = ChessForRigEnv()

obs = env_test.reset()
cumulative_rewards = 0
done = False
i = 0

agentDQN.model.eval()

while not done and i < 300:
    q = agentDQN.model(torch.from_numpy(obs).float().unsqueeze(0).to(device))
    move = int(torch.argmax(q))
    obs, rewards, done, info = env_test.step(move, mode_abandon=False)
    cumulative_rewards += rewards
    i += 1

b = env_test.render(display=False)
print(i, move, cumulative_rewards, b.result(), b.status())
b

In [ ]:
agentDQN.target_model.load_state_dict(torch.load("target_model_dqn_RIG"))
agentDQN.model.load_state_dict(torch.load("model_dqn_RIG"))

#Implementation DQN RIG pour Chess Env
(sur la base du code RCP211/RL/TP3)

In [ ]:
#@title Import des librairies
import os

import random
import gym
import pylab
import numpy as np
from collections import deque
from gym.wrappers import Monitor

#from IPython.display import HTML
#from IPython import display as ipythondisplay
#from pyvirtualdisplay import Display
#display = Display(visible=0, size=(1400, 900))
#display.start()

import time
from matplotlib import pyplot as plt

import pylab as pl
#from IPython import display as ipdisplay

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from numpy import linalg as LA

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#@title Class DQN_RIG_Agent 

class DQN_RIG_Agent(object):
    def __init__(self, env, vae, model, replay1d, update_target_model ,ddqn=False,Soft_Update=False, dueling=False,batch_size=128,gamma=0.95,memory_size=65536, episodes=2000):
              
        self.env = env
        self.vae = vae
        self.env.seed(0)  
        self.replay1d = replay1d
        self.update_target_model = update_target_model

        self.env._max_episode_steps = 4000
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.EPISODES = episodes
        
        # Instantiate memory
        #memory_size = 65536
        self.memory = deque(maxlen=memory_size)

        self.gamma = gamma    # discount rate
        
        # EXPLORATION HYPERPARAMETERS for epsilon and epsilon greedy strategy
        self.epsilon = 1.0  # exploration probability at start
        self.epsilon_min = 0.01  # minimum exploration probability 
        self.epsilon_decay = 0.0005  # exponential decay rate for exploration prob
        
        self.batch_size = batch_size

        # defining model parameters
        self.ddqn = ddqn # use doudle deep q network
        self.Soft_Update = Soft_Update # use soft parameter update
        self.dueling = dueling
        self.epsilon_greedy = False # use epsilon greedy improved strategy
        
        self.TAU = .1 # target network soft update hyperparameter

        self.Save_Path = 'Models'
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.scores, self.episodes, self.average = [], [], []

        self.Model_name = os.path.join(self.Save_Path, "ChessForRig_DQN.h5")
        
        # create main model and target model
        self.model = model(in_channels=self.state_size+latent_dimension,action_space = self.action_size, dueling = False, name = 'online model').to(device)
        self.target_model = model(in_channels=self.state_size+latent_dimension,action_space = self.action_size, dueling = False, name = 'target model').to(device) 

        self.optimizer = optim.RMSprop(self.model.parameters(),lr=1e-3, eps=0.01, alpha=0.95)
        self.criterion = nn.MSELoss()
        self.loss = 0
        self.dd = torch.zeros(64, 4, 5, 5).to(device)
        
    # after some time interval update the target model to be the same as the online model               
                
    def remember(self, state, action, next_state, Zg, reward, done):
        experience = state, action, next_state, Zg, reward, done
        #print(f' state = {state}, action = {action}, reward = {reward}, next_state = {next_state}, done = {done}  ')
        self.memory.append((experience))
    
    def act1d(self, state, decay_step, Z_g):
        # EPSILON GREEDY STRATEGY
        if self.epsilon_greedy:
        # Here we'll use an improved version of our epsilon greedy strategy for Q-learning
            explore_probability = self.epsilon_min + (self.epsilon - self.epsilon_min) * np.exp(-self.epsilon_decay * decay_step)
        # OLD EPSILON STRATEGY
        else:
            if self.epsilon > self.epsilon_min:
                self.epsilon *= (1-self.epsilon_decay)
            explore_probability = self.epsilon
    
        if explore_probability > np.random.rand():
            # Make a random action (exploration)
            #q = random.randrange(self.action_size)
            result = get_random_action(self.env.board)
            action = result.from_square*64+result.to_square
            return action, explore_probability#int(np.argmax(q)), explore_probability

        else:
            # Get action from Q-network (exploitation)
            # Estimate the Qs values state
            # Take the biggest Q value (= the best action)
            q = self.model(torch.from_numpy(state).float().unsqueeze(0).to(device),
                           torch.from_numpy(Z_g).float().to(device))
            return int(torch.argmax(q)), explore_probability
        
    #pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        pylab.plot(self.episodes, self.average, 'r')
        pylab.plot(self.episodes, self.scores, 'b')
        pylab.ylabel('Score', fontsize=18)
        pylab.xlabel('Steps', fontsize=18)
        dqn = 'DQN_'
        softupdate = ''
        dueling = ''
        greedy = ''
        PER = ''
        if self.ddqn: dqn = 'DDQN_'
        if self.Soft_Update: softupdate = '_soft'
        if self.dueling: dueling = '_Dueling'
        if self.epsilon_greedy: greedy = '_Greedy'
        try:
            pylab.savefig(dqn+"ChessForRig"+softupdate+dueling+greedy+PER+"_CNN.png")
        except OSError:
            pass

        return str(self.average[-1])[:5]

    
    def reset1d(self):
        return self.env.reset()

    
    def step1d(self,action):
        next_state, reward, done, info = self.env.step(action)
        return next_state, reward, done, info
    
    def convert_state_flat_to_cnn(self, state):
        s = [np.reshape(state,(6,8,8))]
        s = torch.tensor(s,dtype=torch.float32) 
        return s
    
    def run1d(self):
        decay_step = 0
        loss_tab = []
        
        for e in range(self.EPISODES):
            state = self.reset1d()
            done = False
            i = 0

            cum_rewards = 0

            Zg = torch.randn(1, latent_dimension, device='cpu')

            while not done:
                decay_step += 1
                action, explore_probability = self.act1d(state, decay_step, Zg.numpy())
                next_state, reward, done, _ = self.step1d(action)
                
                #le vae prend une structure matricielle
                #state ou nex_state : (6*8*8,)
                #conversion en (6,8,8)
                #np.reshape(state,(6,8,8))
                z, _ = self.vae.encoder(self.convert_state_flat_to_cnn(state))
                z_prime, _ = self.vae.encoder(self.convert_state_flat_to_cnn(next_state))

                if 0.5 > np.random.rand():
                  Zg_prime = torch.randn(1, latent_dimension, device='cpu')
                else:
                  Zg_prime = Zg

                #calcul reward
                Z = z_prime-Zg_prime
                
                if not done or i == self.env._max_episode_steps-1:
                    reward = reward
                    r = -LA.norm(Z.detach().cpu())
                else:
                    reward = reward
                    if reward == 0: #notre agent n'a pas remporté la partie
                      r = -1e1
                    else:
                      r = 0
                #state, action, next_state, Zg, reward (RIG), done    
                self.remember(state, action, next_state, Zg_prime.detach().cpu(), r, done)
                state = next_state
                i += 1
                cum_rewards += reward

                if done:
                    if e % 10==0:
                        update_target_model(self.Soft_Update,self.ddqn,self.model,self.target_model,self.TAU)
                    average = self.PlotModel(cum_rewards, e)#self.PlotModel(i, e)
                    loss_tab.append(self.loss)
                    print("episode: {}/{}, e: {:.2}, average: {}\n".format(e, self.EPISODES, explore_probability, average))
                    if i == self.env._max_episode_steps:
                        print("Saving trained model to", self.Model_name)
                        break
                self.replay1d(self.env,self.memory,self.batch_size,self.model,self.target_model,self.ddqn,self.gamma,self.criterion,self.loss,self.optimizer)
        plt.plot(loss_tab)

In [ ]:
#@title class DQN_RIG_1d pour fonction valeur
class DQN_RIG_1d(nn.Module):

    def __init__(self, action_space, in_channels, dueling=False, name='NN'):
        super(DQN_RIG_1d, self).__init__()
        self.name = name
        self.dueling = dueling
        self.action_space = action_space

        self.head1 = nn.Linear(in_channels, 512)
        self.head2 = nn.Linear(512, 256)
        self.head3 = nn.Linear(256, 64)
        
        if dueling:
            self.state_value = nn.Linear(64,1)
            self.action_advantage = nn.Linear(64,action_space)
        else:
            self.Q = nn.Linear(64,action_space)
            
        
    def forward(self, x, z_g):

        #print(f'{self.name} x_size = {x.shape}, z_g_size = {z_g.shape}')

        x = torch.concat((x, z_g), dim=1)
        
        x = F.relu(self.head1(x))
        x = F.relu(self.head2(x))
        x = F.relu(self.head3(x))
        
        if self.dueling:
            s = self.state_value(x).repeat((1,self.action_space))
            a = self.action_advantage(x)
            a -= torch.mean(a,dim=1, keepdim=True).repeat(1,2)
            x = s+a
        else:
            x = self.Q(x)       
        
        #print(f'{self.name} output = {x.size()}')
        return x


In [ ]:
#@title replay_RIG_1d
def replay_RIG_1d(env,memory,batch_size,model,target_model,ddqn,gamma,criterion,loss,optimizer):
        # Randomly sample minibatch from the deque memory
        # memory : state, action, next_state, Zg, reward, done
        #          0      1       2           3   4       5
        minibatch = random.sample(memory, min(len(memory), batch_size))

        state = np.zeros([batch_size,env.observation_space.shape[0]])
        #print(state.shape)
        next_state = np.zeros([batch_size,env.observation_space.shape[0]])
        z_goals = np.zeros([batch_size,latent_dimension])
        action, reward, done = [], [], []

        # do this before prediction
        # for speedup, this could be done on the tensor level
        # but easier to understand using a loop       
        for i in range(len(minibatch)):
            #print("minibatch[i][0] = ",minibatch[i][0])
            state[i] = minibatch[i][0]
            z_goals[i] = minibatch[i][3]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][4])
            next_state[i] = minibatch[i][2]
            done.append(minibatch[i][5])
        # do batch prediction to save speed
        # predict Q-values for starting state using the main network
        target = model(torch.from_numpy(state).float().to(device),torch.from_numpy(z_goals).float().to(device)).detach()
        target_old = target.clone()
        # predict best action in ending state using the main network
        target_next = model(torch.from_numpy(next_state).float().to(device),torch.from_numpy(z_goals).float().to(device)).detach()
        # predict Q-values for ending state using the target network
        target_val = target_model(torch.from_numpy(next_state).float().to(device),torch.from_numpy(z_goals).float().to(device)).detach()
        for i in range(len(minibatch)):
            # correction on the Q value for the action used
            if done[i]:
                target[i][action[i]] = torch.tensor(reward[i])
            else:
                # the key point of Double DQN
                # selection of action is from model
                # update is from target model
                if ddqn: # Double - DQN
                    # current Q Network selects the action
                    # a'_max = argmax_a' Q(s', a')
                    a =  int(torch.argmax(target_next[i]))# np.argmax(target_next[i])
                    # target Q Network evaluates the action
                    # Q_max = Q_target(s', a'_max)
                    target[i][action[i]] = reward[i] + gamma * (target_val[i][a])
                else: # Standard - DQN
                    # DQN chooses the max Q value among next actions
                    # selection and evaluation of action is on the target Q Network
                    # Q_max = max_a' Q_target(s', a')
                    target[i][action[i]] = reward[i] + gamma * target_next[i].max(0)[0].detach()

            
                
        # Train the Neural Network with batches
        # self.model.fit(state, target, batch_size=self.batch_size, verbose=0)
        pred = model(torch.from_numpy(state).float().to(device),torch.from_numpy(z_goals).float().to(device))
        loss = criterion(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
#@title update_target_model_RIG
def update_target_model_RIG(Soft_Update,ddqn,model,target_model,TAU):
        if not Soft_Update:
            
            target_model.load_state_dict(model.state_dict()) 
            return
        if Soft_Update and ddqn:    
            with torch.no_grad():
                #dict_model = dict(self.model.named_parameters())
                #dict_target = dict(self.target.named_paramters())
                for model, target in zip(model.named_parameters(), target_model.named_parameters()):
                    model_name, model_weight = model
                    target_name, target_weight = target
                    #print(f'target_weight = {target_weight.data}')
                    tmp = target_weight.data * (1-TAU) + model_weight.data * TAU
                    target_weight.data.copy_(tmp)


##Run DQN RIG dans l'environnement Chess

In [ ]:
env = ChessForRigEnv()

In [ ]:
agentDQN_RIG = DQN_RIG_Agent(env, vae, DQN_RIG_1d, replay_RIG_1d,update_target_model_RIG, 
                             episodes=2000,gamma=0.99,ddqn=True,Soft_Update=True)

Remplir la mémoire de l'expérience replay avec des observations de trajectoires de jeux gagnantes

In [ ]:
rounds = 500
max_iter_per_round = 1000

engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish-5-linux/Linux/stockfish_14053109_x64")

env_fill_replay = ChessForRigEnv()

nb_in_mem = 0

for i in range(rounds):
  state = env_fill_replay.reset()
  done = False
  k = 0
  while not done and k < max_iter_per_round:
    result = engine.play(env_fill_replay.board, chess.engine.Limit(time=0.1, depth=4, nodes=3))
    result = result.move
    action = result.from_square*64+result.to_square
    next_state, reward, done, info = env_fill_replay.step(action)
    
    z, _ = agentDQN_RIG.vae.encoder(agentDQN_RIG.convert_state_flat_to_cnn(state))
    z_prime, _ = agentDQN_RIG.vae.encoder(agentDQN_RIG.convert_state_flat_to_cnn(next_state))
    Z = z-z_prime
    r = -LA.norm(Z.detach().cpu())
    agentDQN_RIG.remember(state, action, next_state, z_prime.detach().cpu(), r, done)
    state = next_state
    k += 1
    nb_in_mem += 1

engine.quit()

In [ ]:
agentDQN_RIG.model.train()
agentDQN_RIG.run1d()

Test en production du DQN RIG

In [ ]:
obs_a_atteindre = np.reshape(board_to_numpy(board_gagnant),(6*8*8,))
z_but, _ = agentDQN_RIG.vae.encoder(agentDQN_RIG.convert_state_flat_to_cnn(obs_a_atteindre))

In [ ]:
torch.save(agentDQN.target_model.state_dict(), "target_model_dqn_RIG")
torch.save(agentDQN.model.state_dict(), "model_dqn_RIG")

In [ ]:
env_test = ChessForRigEnv()

obs = env_test.reset()
cumulative_rewards = 0
done = False
i = 0

agentDQN_RIG.model.eval()

while not done and i < 300:
    q = agentDQN_RIG.model(torch.from_numpy(obs).float().unsqueeze(0).to(device),
                           z_but.to(device))
    move = int(torch.argmax(q))
    obs, rewards, done, info = env_test.step(move, mode_abandon=False)
    cumulative_rewards += rewards
    i += 1

b = env_test.render(display=False)
print(i, move, cumulative_rewards, b.result(), b.status())
b

In [ ]:
agentDQN_RIG.target_model.load_state_dict(torch.load("target_model_dqn_RIG"))
agentDQN_RIG.model.load_state_dict(torch.load("model_dqn_RIG"))

#Tests RLC "capture"

##RLC "capture"
[Reprise du code de la librairie RLC de arjangroen](https://github.com/arjangroen/RLC/tree/master/RLC/capture_chess)

In [ ]:
#@title Agent
import tensorflow as tf
import keras

from keras.models import Model, clone_model
from keras.layers import Input, Conv2D, Dense, Reshape, Dot, Activation, Multiply
from tensorflow.keras.optimizers import SGD

import numpy as np
import keras.backend as K


def policy_gradient_loss(Returns):
    def modified_crossentropy(action, action_probs):
        cost = (K.categorical_crossentropy(action, action_probs, from_logits=False, axis=1) * Returns)
        return K.mean(cost)

    return modified_crossentropy


class Agent(object):

    def __init__(self, gamma=0.5, network='linear', lr=0.01, verbose=0):
        """
        Agent that plays the white pieces in capture chess
        Args:
            gamma: float
                Temporal discount factor
            network: str
                'linear' or 'conv'
            lr: float
                Learning rate, ideally around 0.1
        """
        self.gamma = gamma
        self.network = network
        self.lr = lr
        self.verbose = verbose
        self.init_network()
        self.weight_memory = []
        self.long_term_mean = []

    def init_network(self):
        """
        Initialize the network
        Returns:

        """
        if self.network == 'linear':
            self.init_linear_network()
        elif self.network == 'conv':
            self.init_conv_network()
        elif self.network == 'conv_pg':
            self.init_conv_pg()

    def fix_model(self):
        """
        The fixed model is the model used for bootstrapping
        Returns:
        """
        optimizer = SGD(lr=self.lr, momentum=0.0, decay=0.0, nesterov=False)
        self.fixed_model = clone_model(self.model)
        self.fixed_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        self.fixed_model.set_weights(self.model.get_weights())

    def init_linear_network(self):
        """
        Initialize a linear neural network
        Returns:

        """
        optimizer = SGD(lr=self.lr, momentum=0.0, decay=0.0, nesterov=False)
        input_layer = Input(shape=(8, 8, 8), name='board_layer')
        reshape_input = Reshape((512,))(input_layer)
        output_layer = Dense(4096)(reshape_input)
        self.model = Model(inputs=[input_layer], outputs=[output_layer])
        self.model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    def init_conv_network(self):
        """
        Initialize a convolutional neural network
        Returns:

        """
        optimizer = SGD(lr=self.lr, momentum=0.0, decay=0.0, nesterov=False)
        input_layer = Input(shape=(8, 8, 8), name='board_layer')
        inter_layer_1 = Conv2D(1, (1, 1), data_format="channels_first")(input_layer)  # 1,8,8
        inter_layer_2 = Conv2D(1, (1, 1), data_format="channels_first")(input_layer)  # 1,8,8
        flat_1 = Reshape(target_shape=(1, 64))(inter_layer_1)
        flat_2 = Reshape(target_shape=(1, 64))(inter_layer_2)
        output_dot_layer = Dot(axes=1)([flat_1, flat_2])
        output_layer = Reshape(target_shape=(4096,))(output_dot_layer)
        self.model = Model(inputs=[input_layer], outputs=[output_layer])
        self.model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    def init_conv_pg(self):
        """
        Convnet net for policy gradients
        Returns:

        """
        optimizer = SGD(lr=self.lr, momentum=0.0, decay=0.0, nesterov=False)
        input_layer = Input(shape=(8, 8, 8), name='board_layer')
        R = Input(shape=(1,), name='Rewards')
        legal_moves = Input(shape=(4096,), name='legal_move_mask')
        inter_layer_1 = Conv2D(1, (1, 1), data_format="channels_first")(input_layer)  # 1,8,8
        inter_layer_2 = Conv2D(1, (1, 1), data_format="channels_first")(input_layer)  # 1,8,8
        flat_1 = Reshape(target_shape=(1, 64))(inter_layer_1)
        flat_2 = Reshape(target_shape=(1, 64))(inter_layer_2)
        output_dot_layer = Dot(axes=1)([flat_1, flat_2])
        output_layer = Reshape(target_shape=(4096,))(output_dot_layer)
        softmax_layer = Activation('softmax')(output_layer)
        legal_softmax_layer = Multiply()([legal_moves, softmax_layer])  # Select legal moves
        self.model = Model(inputs=[input_layer, R, legal_moves], outputs=[legal_softmax_layer])
        self.model.compile(optimizer=optimizer, loss=policy_gradient_loss(R))

    def network_update(self, minibatch):
        """
        Update the Q-network using samples from the minibatch
        Args:
            minibatch: list
                The minibatch contains the states, moves, rewards and new states.

        Returns:
            td_errors: np.array
                array of temporal difference errors

        """

        # Prepare separate lists
        states, moves, rewards, new_states = [], [], [], []
        td_errors = []
        episode_ends = []
        for sample in minibatch:
            states.append(sample[0])
            moves.append(sample[1])
            rewards.append(sample[2])
            new_states.append(sample[3])

            # Episode end detection
            if np.array_equal(sample[3], sample[3] * 0):
                episode_ends.append(0)
            else:
                episode_ends.append(1)

        # The Q target
        q_target = np.array(rewards) + np.array(episode_ends) * self.gamma * np.max(
            self.fixed_model.predict(np.stack(new_states, axis=0)), axis=1)

        # The Q value for the remaining actions
        q_state = self.model.predict(np.stack(states, axis=0))  # batch x 64 x 64

        # Combine the Q target with the other Q values.
        q_state = np.reshape(q_state, (len(minibatch), 64, 64))
        for idx, move in enumerate(moves):
            td_errors.append(q_state[idx, move[0], move[1]] - q_target[idx])
            q_state[idx, move[0], move[1]] = q_target[idx]
        q_state = np.reshape(q_state, (len(minibatch), 4096))

        # Perform a step of minibatch Gradient Descent.
        self.model.fit(x=np.stack(states, axis=0), y=q_state, epochs=1, verbose=0)

        return td_errors

    def get_action_values(self, state):
        """
        Get action values of a state
        Args:
            state: np.ndarray with shape (8,8,8)
                layer_board representation

        Returns:
            action values

        """
        return self.fixed_model.predict(state) + np.random.randn() * 1e-9

    def policy_gradient_update(self, states, actions, rewards, action_spaces, actor_critic=False):
        """
        Update parameters with Monte Carlo Policy Gradient algorithm
        Args:
            states: (list of tuples) state sequence in episode
            actions: action sequence in episode
            rewards: rewards sequence in episode

        Returns:

        """
        n_steps = len(states)
        Returns = []
        targets = np.zeros((n_steps, 64, 64))
        for t in range(n_steps):
            action = actions[t]
            targets[t, action[0], action[1]] = 1
            if actor_critic:
                R = rewards[t, action[0] * 64 + action[1]]
            else:
                R = np.sum([r * self.gamma ** i for i, r in enumerate(rewards[t:])])
            Returns.append(R)

        if not actor_critic:
            mean_return = np.mean(Returns)
            self.long_term_mean.append(mean_return)
            train_returns = np.stack(Returns, axis=0) - np.mean(self.long_term_mean)
        else:
            train_returns = np.stack(Returns, axis=0)
        # print(train_returns.shape)
        targets = targets.reshape((n_steps, 4096))
        self.weight_memory.append(self.model.get_weights())
        self.model.fit(x=[np.stack(states, axis=0),
                          train_returns,
                          np.concatenate(action_spaces, axis=0)
                          ],
                       y=[np.stack(targets, axis=0)],
                       verbose=self.verbose
                       )

In [ ]:
#@title Environnement
import chess
import chess.engine
import numpy as np

mapper = {}
mapper["p"] = 0
mapper["r"] = 1
mapper["n"] = 2
mapper["b"] = 3
mapper["q"] = 4
mapper["k"] = 5
mapper["P"] = 0
mapper["R"] = 1
mapper["N"] = 2
mapper["B"] = 3
mapper["Q"] = 4
mapper["K"] = 5


class Board(object):

    def __init__(self, FEN=None, victory_reward=0):
        """
        Chess Board Environment
        Args:
            FEN: str
                Starting FEN notation, if None then start in the default chess position
        """
        self.FEN = FEN
        self.board = chess.Board(self.FEN) if self.FEN else chess.Board()
        self.init_action_space()
        self.layer_board = np.zeros(shape=(8, 8, 8))
        self.init_layer_board()
        self.victory_reward = victory_reward

    def init_action_space(self):
        """
        Initialize the action space
        Returns:
        """
        self.action_space = np.zeros(shape=(64, 64))

    def init_layer_board(self):
        """
        Initalize the numerical representation of the environment
        Returns:
        """
        self.layer_board = np.zeros(shape=(8, 8, 8))
        for i in range(64):
            row = i // 8
            col = i % 8
            piece = self.board.piece_at(i)
            if piece == None:
                continue
            elif piece.symbol().isupper():
                sign = 1
            else:
                sign = -1
            layer = mapper[piece.symbol()]
            self.layer_board[layer, row, col] = sign
        if self.board.turn:
            self.layer_board[6, :, :] = 1 / self.board.fullmove_number
        if self.board.can_claim_draw():
            self.layer_board[7, :, :] = 1

    def step(self, action, opponent=None):
        """
        Run a step
        Args:
            action: tuple of 2 integers
                Move from, Move to
        Returns:
            epsiode end: Boolean
                Whether the episode has ended
            reward: int
                Difference in material value after the move
        """
        piece_balance_before = self.get_material_value()
        self.board.push(action)
        self.init_layer_board()
        piece_balance_after = self.get_material_value()
        if self.board.result() == "*":
            if opponent == None:
              opponent_move = self.get_random_action()
            else:
              PlayResult = opponent.play(self.board, chess.engine.Limit(time=0.1, depth=4, nodes=4))
              opponent_move = PlayResult.move
            self.board.push(opponent_move)
            self.init_layer_board() 
            capture_reward = piece_balance_after - piece_balance_before
            if self.board.result() == "*":
                reward = 0 + capture_reward
                episode_end = False
            else:
                if self.board.result() == "1-0":
                  reward = self.victory_reward + capture_reward
                else:
                  reward = 0 + capture_reward
                episode_end = True
        else:
            capture_reward = piece_balance_after - piece_balance_before
            if self.board.result() == "1-0":
                  reward = self.victory_reward + capture_reward
            else:
                  reward = 0 + capture_reward
            episode_end = True
        if self.board.is_game_over():
            reward = 0
            episode_end = True
        return episode_end, reward

    def get_random_action(self):
        """
        Sample a random action
        Returns: move
            A legal python chess move.
        """
        legal_moves = [x for x in self.board.generate_legal_moves()]
        legal_moves = np.random.choice(legal_moves)
        return legal_moves

    def project_legal_moves(self):
        """
        Create a mask of legal actions
        Returns: np.ndarray with shape (64,64)
        """
        self.action_space = np.zeros(shape=(64, 64))
        moves = [[x.from_square, x.to_square] for x in self.board.generate_legal_moves()]
        for move in moves:
            self.action_space[move[0], move[1]] = 1
        return self.action_space

    def get_material_value(self):
        """
        Sums up the material balance using Reinfield values
        Returns: The material balance on the board
        """
        pawns = 1 * np.sum(self.layer_board[0, :, :])
        rooks = 5 * np.sum(self.layer_board[1, :, :])
        minor = 3 * np.sum(self.layer_board[2:4, :, :])
        queen = 9 * np.sum(self.layer_board[4, :, :])
        return pawns + rooks + minor + queen

    def reset(self):
        """
        Reset the environment
        Returns:
        """
        self.board = chess.Board(self.FEN) if self.FEN else chess.Board()
        self.init_layer_board()
        self.init_action_space()

In [ ]:
#@title Learn
import numpy as np
from chess.pgn import Game
import pandas as pd


class Q_learning(object):

    def __init__(self, agent, env, memsize=1000):
        """
        Reinforce object to learn capture chess
        Args:
            agent: The agent playing the chess game as white
            env: The environment including the python-chess board
            memsize: maximum amount of games to retain in-memory
        """
        self.agent = agent
        self.env = env
        self.memory = []
        self.memsize = memsize
        self.reward_trace = []
        self.memory = []
        self.sampling_probs = []

    def learn(self, iters=100, c=10, windows_r_average=10):
        """
        Run the Q-learning algorithm. Play greedy on the final iter
        Args:
            iters: int
                amount of games to train
            c: int
                update the network every c games

        Returns: pgn (str)
            pgn string describing final game

        """
        for k in range(iters):
            greedy = True if k == iters - 1 else False
            self.env.reset()
            self.play_game(k, greedy=greedy)
            if k % c == 0:
                all_r = np.array(self.reward_trace)
                r_average = all_r[-windows_r_average:].mean()
                print("iter", k, " r_average", r_average)
                self.agent.fix_model()

        pgn = Game.from_board(self.env.board)
        reward_smooth = pd.DataFrame(self.reward_trace)
        reward_smooth.rolling(window=10, min_periods=0).mean().plot()

        return pgn

    def play_game(self, k, greedy=False, maxiter=25):
        """
        Play a game of capture chess
        Args:
            k: int
                game count, determines epsilon (exploration rate)
            greedy: Boolean
                if greedy, no exploration is done
            maxiter: int
                Maximum amount of steps per game

        Returns:

        """
        episode_end = False
        turncount = 0

        # Here we determine the exploration rate. k is divided by 250 to slow down the exploration rate decay.
        eps = max(0.05, 1 / (1 + (k / 250))) if not greedy else 0.

        # Play a game of chess
        while not episode_end:
            state = self.env.layer_board
            explore = np.random.uniform(0, 1) < eps  # determine whether to explore
            if explore:
                move = self.env.get_random_action()
                move_from = move.from_square
                move_to = move.to_square
            else:
                action_values = self.agent.get_action_values(np.expand_dims(state, axis=0))
                action_values = np.reshape(np.squeeze(action_values), (64, 64))
                action_space = self.env.project_legal_moves()  # The environment determines which moves are legal
                action_values = np.multiply(action_values, action_space)
                move_from = np.argmax(action_values, axis=None) // 64
                move_to = np.argmax(action_values, axis=None) % 64
                moves = [x for x in self.env.board.generate_legal_moves() if \
                         x.from_square == move_from and x.to_square == move_to]
                if len(moves) == 0:  # If all legal moves have negative action value, explore.
                    move = self.env.get_random_action()
                    move_from = move.from_square
                    move_to = move.to_square
                else:
                    move = np.random.choice(moves)  # If there are multiple max-moves, pick a random one.

            episode_end, reward = self.env.step(move)
            new_state = self.env.layer_board
            if len(self.memory) > self.memsize:
                self.memory.pop(0)
                self.sampling_probs.pop(0)
            turncount += 1
            if turncount > maxiter:
                episode_end = True
                reward = 0
            if episode_end:
                new_state = new_state * 0
            self.memory.append([state, (move_from, move_to), reward, new_state])
            self.sampling_probs.append(1)

            self.reward_trace.append(reward)

            self.update_agent(turncount)

        return self.env.board

    def sample_memory(self, turncount):
        """
        Get a sample from memory for experience replay
        Args:
            turncount: int
                turncount limits the size of the minibatch

        Returns: tuple
            a mini-batch of experiences (list)
            indices of chosen experiences

        """
        minibatch = []
        memory = self.memory[:-turncount]
        probs = self.sampling_probs[:-turncount]
        sample_probs = [probs[n] / np.sum(probs) for n in range(len(probs))]
        indices = np.random.choice(range(len(memory)), min(1028, len(memory)), replace=True, p=sample_probs)
        for i in indices:
            minibatch.append(memory[i])

        return minibatch, indices

    def update_agent(self, turncount):
        """
        Update the agent using experience replay. Set the sampling probs with the td error
        Args:
            turncount: int
                Amount of turns played. Only sample the memory of there are sufficient samples
        Returns:

        """
        if turncount < len(self.memory):
            minibatch, indices = self.sample_memory(turncount)
            td_errors = self.agent.network_update(minibatch)
            for n, i in enumerate(indices):
                self.sampling_probs[i] = np.abs(td_errors[n])


class Reinforce(object):

    def __init__(self, agent, env, opponent=None):
        """
        Reinforce object to learn capture chess
        Args:
            agent: The agent playing the chess game as white
            env: The environment including the python-chess board
            memsize: maximum amount of games to retain in-memory
        """
        self.agent = agent
        self.env = env
        self.reward_trace = []
        self.action_value_mem = []
        self.opponent = opponent

    def learn(self, iters=100, c=10):
        """
        Run the Q-learning algorithm. Play greedy on the final iter
        Args:
            iters: int
                amount of games to train
            c: int
                update the network every c games

        Returns: pgn (str)
            pgn string describing final game

        """
        for k in range(iters):
            self.env.reset()
            states, actions, rewards, action_spaces = self.play_game(k)
            self.reinforce_agent(states, actions, rewards, action_spaces)

        pgn = Game.from_board(self.env.board)
        reward_smooth = pd.DataFrame(self.reward_trace)
        reward_smooth.rolling(window=10, min_periods=0).mean().plot()

        return pgn

    def play_game(self, k, maxiter=25):
        """
        Play a game of capture chess
        Args:
            k: int
                game count, determines epsilon (exploration rate)
            greedy: Boolean
                if greedy, no exploration is done
            maxiter: int
                Maximum amount of steps per game

        Returns:

        """
        episode_end = False
        turncount = 0

        states = []
        actions = []
        rewards = []
        action_spaces = []

        # Play a game of chess
        while not episode_end:
            state = self.env.layer_board
            action_space = self.env.project_legal_moves()  # The environment determines which moves are legal
            action_probs = self.agent.model.predict([np.expand_dims(state, axis=0),
                                                     np.zeros((1, 1)),
                                                     action_space.reshape(1, 4096)])
            self.action_value_mem.append(action_probs)
            action_probs = action_probs / action_probs.sum()
            move = np.random.choice(range(4096), p=np.squeeze(action_probs))
            move_from = move // 64
            move_to = move % 64
            moves = [x for x in self.env.board.generate_legal_moves() if \
                     x.from_square == move_from and x.to_square == move_to]
            assert len(moves) > 0  # should not be possible
            if len(moves) > 1:
                move = np.random.choice(moves)  # If there are multiple max-moves, pick a random one.
            elif len(moves) == 1:
                move = moves[0]

            episode_end, reward = self.env.step(move, self.opponent)
            new_state = self.env.layer_board
            turncount += 1
            if turncount > maxiter:
                episode_end = True
                reward = 0
            if episode_end:
                new_state = new_state * 0

            states.append(state)
            actions.append((move_from, move_to))
            rewards.append(reward)
            action_spaces.append(action_space.reshape(1, 4096))

        self.reward_trace.append(np.sum(rewards))

        return states, actions, rewards, action_spaces

    def reinforce_agent(self, states, actions, rewards, action_spaces):
        """
        Update the agent using experience replay. Set the sampling probs with the td error
        Args:
            turncount: int
                Amount of turns played. Only sample the memory of there are sufficient samples
        Returns:

        """
        self.agent.policy_gradient_update(states, actions, rewards, action_spaces)


class ActorCritic(object):

    def __init__(self, actor, critic, env):
        """
        ActorCritic object to learn capture chess
        Args:
            actor: Policy Gradient Agent
            critic: Q-learning Agent
            env: The environment including the python-chess board
            memsize: maximum amount of games to retain in-memory
        """
        self.actor = actor
        self.critic = critic
        self.env = env
        self.reward_trace = []
        self.action_value_mem = []
        self.memory = []
        self.sampling_probs = []

    def learn(self, iters=100, c=10):
        """
        Run the Q-learning algorithm. Play greedy on the final iter
        Args:
            iters: int
                amount of games to train
            c: int
                update the network every c games

        Returns: pgn (str)
            pgn string describing final game

        """
        for k in range(iters):
            if k % c == 0:
                self.critic.fix_model()
            self.env.reset()
            end_state = self.play_game(k)

        pgn = Game.from_board(self.env.board)
        reward_smooth = pd.DataFrame(self.reward_trace)
        reward_smooth.rolling(window=10, min_periods=0).mean().plot()

        return pgn

    def play_game(self, k, greedy=False, maxiter=25):
        """
        Play a game of capture chess
        Args:
            k: int
                game count, determines epsilon (exploration rate)
            greedy: Boolean
                if greedy, no exploration is done
            maxiter: int
                Maximum amount of steps per game

        Returns:

        """
        episode_end = False
        turncount = 0

        # Play a game of chess
        state = self.env.layer_board
        while not episode_end:
            state = self.env.layer_board
            action_space = self.env.project_legal_moves()  # The environment determines which moves are legal
            action_probs = self.actor.model.predict([np.expand_dims(state, axis=0),
                                                     np.zeros((1, 1)),
                                                     action_space.reshape(1, 4096)])
            self.action_value_mem.append(action_probs)
            # print(action_probs)
            # print(np.max(action_probs))
            action_probs = action_probs / action_probs.sum()
            move = np.random.choice(range(4096), p=np.squeeze(action_probs))
            move_from = move // 64
            move_to = move % 64
            moves = [x for x in self.env.board.generate_legal_moves() if \
                     x.from_square == move_from and x.to_square == move_to]
            assert len(moves) > 0  # should not be possible
            if len(moves) > 1:
                move = np.random.choice(moves)  # If there are multiple max-moves, pick a random one.
            elif len(moves) == 1:
                move = moves[0]

            episode_end, reward = self.env.step(move)
            new_state = self.env.layer_board
            turncount += 1
            if turncount > maxiter:
                episode_end = True
                reward = 0
            if episode_end:
                new_state = new_state * 0

            self.memory.append([state, (move_from, move_to), reward, new_state, action_space.reshape(1, 4096)])
            self.sampling_probs.append(1)
            self.reward_trace.append(reward)

        self.update_actorcritic(turncount)

        return self.env.board

    def sample_memory(self, turncount):
        """
        Get a sample from memory for experience replay
        Args:
            turncount: int
                turncount limits the size of the minibatch

        Returns: tuple
            a mini-batch of experiences (list)
            indices of chosen experiences

        """
        minibatch = []
        memory = self.memory[:-turncount]
        probs = self.sampling_probs[:-turncount]
        sample_probs = [probs[n] / np.sum(probs) for n in range(len(probs))]
        indices = np.random.choice(range(len(memory)), min(1028, len(memory)), replace=False, p=sample_probs)
        for i in indices:
            minibatch.append(memory[i])

        return minibatch, indices

    def update_actorcritic(self, turncount):
        """Actor critic"""

        if turncount < len(self.memory):

            # Get a sampple
            minibatch, indices = self.sample_memory(turncount)

            # Update critic and find td errors for prioritized experience replay
            td_errors = self.critic.network_update(minibatch)

            # Get a Q value from the critic
            states = [x[0] for x in minibatch]
            actions = [x[1] for x in minibatch]
            Q_est = self.critic.get_action_values(np.stack(states, axis=0))
            action_spaces = [x[4] for x in minibatch]

            self.actor.policy_gradient_update(states, actions, Q_est, action_spaces, actor_critic=True)

            # Update sampling probs
            for n, i in enumerate(indices):
                self.sampling_probs[i] = np.abs(td_errors[n])

    def update_critic(self, turncount):
        """
        Update the agent using experience replay. Set the sampling probs with the td error
        Args:
            turncount: int
                Amount of turns played. Only sample the memory of there are sufficient samples
        Returns:

        """
        if turncount < len(self.memory):
            minibatch, indices = self.sample_memory(turncount)
            td_errors = self.critic.network_update(minibatch)

            for n, i in enumerate(indices):
                self.sampling_probs[i] = np.abs(td_errors[n])

##Apprentissage Agents RL Reinforce

In [ ]:
import tensorflow._api.v2.compat.v1 as tf
import time

import chess.engine

engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish-5-linux/Linux/stockfish_14053109_x64")

tf.disable_v2_behavior()

board = Board(victory_reward=5)
agent = Agent(network='conv_pg',lr=0.1)
R = Reinforce(agent,board)#, opponent=engine)

start = time.time()
pgn = R.learn(iters=8000)
end = time.time()
print(end - start)

engine.quit()

##Parties agent RLC (Reinforce) contre lui même

In [ ]:
def run_games_2(rounds=1000):
  round_success_w = 0
  round_success_b = 0
  for i in range(rounds):
      max_iter = 1000
      k = 0
      board = chess.Board()
      while not board.is_game_over() and k < max_iter:
          k += 1
          if(board.turn): # blanc, notre agent joue
            white_move = next_white_move(agent, board, debug=False, best_probs=True)
            board.push(white_move)
          else: # black, c'est encore le même agent qui joue
            result = next_white_move(agent, board, debug=False, best_probs=True)
            board.push(result)
      success_w = 1 if board.result()=="1-0" else 0
      success_b = 1 if board.result()=="0-1" else 0
      round_success_w += success_w
      round_success_b += success_b
  return round_success_w/rounds, round_success_b/rounds

rounds = 1000

round_success_w, round_success_b = run_games_2(rounds=rounds)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.style as style

style.use('seaborn-pastel')

Class = ["1"]
w = 0.6
round_success_nul = 1 - (round_success_w + round_success_b)

fig = plt.figure(figsize=(12,8))

color = 'steelblue'
plt.bar(Class, round_success_w, w, label='blanc gagnant', color = color,alpha=0.3)
plt.bar(Class, round_success_nul, w, bottom=round_success_w, label='match nul', color = color, alpha=0.7)
plt.bar(Class, round_success_b, w, bottom=round_success_nul+round_success_w, label='noir gagnant', color = color)
plt.xlabel("")
plt.ylabel("Répartition des victoires")
plt.title("Résultats RLC Reinforce vs RLC Reinforce - 1000 parties")
plt.legend()
plt.show()

print(round_success_w,round_success_nul, round_success_b)

##Parties agent RLC (Reinforce) contre agent aléatoire

In [ ]:
def run_games_3(rounds=1000):
  round_success_w = 0
  round_success_b = 0
  for i in range(rounds):
      max_iter = 1000
      k = 0
      board = chess.Board()
      while not board.is_game_over() and k < max_iter:
          k += 1
          if(board.turn): # blanc, un agent aléatoire pur
            white_move = get_random_action(board)
            board.push(white_move)
          else: # black, c'est notre agent (un petit désavantage)
            result = next_white_move(agent, board, debug=False, best_probs=True)
            board.push(result)
      success_w = 1 if board.result()=="1-0" else 0
      success_b = 1 if board.result()=="0-1" else 0
      round_success_w += success_w
      round_success_b += success_b
  return round_success_w/rounds, round_success_b/rounds

rounds = 1000

round_success_w, round_success_b = run_games_3(rounds=rounds)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.style as style

style.use('seaborn-pastel')

Class = ["1"]
w = 0.6
round_success_nul = 1 - (round_success_w + round_success_b)

fig = plt.figure(figsize=(12,8))

color = 'steelblue'
plt.bar(Class, round_success_w, w, label='blanc gagnant', color = color,alpha=0.3)
plt.bar(Class, round_success_nul, w, bottom=round_success_w, label='match nul', color = color, alpha=0.7)
plt.bar(Class, round_success_b, w, bottom=round_success_nul+round_success_w, label='noir gagnant', color = color)
plt.xlabel("")
plt.ylabel("Répartition des victoires")
plt.title("Résultats RLC Reinforce vs aléatoire - 1000 parties")
plt.legend()
plt.show()

print(round_success_w,round_success_nul, round_success_b)

##Parties avec Agent RLC (Reinforce) en blanc et stockfish en noir

In [ ]:
import chess
import chess.engine

rounds = 1000

engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish-5-linux/Linux/stockfish_14053109_x64")

def run_games(rounds=1000, time=0.1, depth=2, nodes=3):
  round_success_w = 0
  round_success_b = 0
  for i in range(rounds):
      max_iter = 1000
      k = 0
      board = chess.Board()
      while not board.is_game_over() and k < max_iter:
          k += 1
          if(board.turn): # blanc, notre agent joue
            white_move = next_white_move(agent, board, debug=False, best_probs=True)
            board.push(white_move)
          else: # black, c'est stockfish qui joue
            result = engine.play(board, chess.engine.Limit(time=time, depth=depth, nodes=nodes))
            board.push(result.move)
      success_w = 1 if board.result()=="1-0" else 0
      success_b = 1 if board.result()=="0-1" else 0
      round_success_w += success_w
      round_success_b += success_b
  return round_success_w/rounds, round_success_b/rounds

n = 4
round_success_w = np.zeros(n)
round_success_b = np.zeros(n)

for i in range(n):
  round_success_w[i], round_success_b[i] = run_games(rounds=rounds, time=0.1, depth=4, nodes=i+1)

engine.quit()

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci(r"./stockfish-5-linux/Linux/stockfish_14053109_x64")
round_success_w_10_noeuds, round_success_b_10_noeuds = run_games(rounds=rounds, time=0.1, depth=4, nodes=10)
engine.quit()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.style as style

style.use('seaborn-pastel')

Class = ["1 noeud", "2 noeuds", "3 noeuds", "4 noeuds"]
w = 0.6
round_success_nul = 1 - (round_success_w + round_success_b)

fig = plt.figure(figsize=(12,8))

color = 'steelblue'
plt.bar(Class, round_success_w, w, label='blanc gagnant', color = color,alpha=0.3)
plt.bar(Class, round_success_nul, w, bottom=round_success_w, label='match nul', color = color, alpha=0.7)
plt.bar(Class, round_success_b, w, bottom=round_success_nul+round_success_w, label='noir gagnant', color = color)
plt.xlabel("Limites stockfish - noeuds (profondeur commune = 4)")
plt.ylabel("Répartition des victoires")
plt.title("Résultats RLC Reinforce vs Stockfish (limité) - 1000 parties")
plt.legend()
plt.show()

print(round_success_w,round_success_nul, round_success_b)